In [8]:
from spacy import tokenizer
from spacy.lang.en import English
from  spacy.matcher import Matcher, PhraseMatcher
import spacy
import re
import cupy

In [9]:
#source of en_core_web_sm is from OntoNotes 5
nlp = spacy.load('en_core_web_sm')
#source of en_core_web_md is from  OntoNotes 5 and GloVe Common Crawl 
nlp_large = spacy.load('en_core_web_md')

In [10]:

doc = nlp("""Modern natural language processing
(NLP) research requires writing code.
Ideally this code would provide a precise definition of the approach, easy
repeatability of results, and a basis for
extending the research. However, many
research codebases bury high-level parameters under implementation details,
are challenging to run and debug, and are
difficult enough to extend that they are
more likely to be rewritten. This paper
describes AllenNLP, a library for applying
deep learning methods to NLP research,
which addresses these issues with easy-to-use command-line tools, declarative
configuration-driven experiments, and
modular NLP abstractions. AllenNLP
has already increased the rate of research
experimentation and the sharing of NLP
components at the Allen Institute for
Artificial Intelligence, and we are working
to have the same impact across the field.
""")

<h1 style="color:red">Tokenizing</h1>

In [13]:
doc_1 = nlp("Modern natural language processing (NLP) research requires writing code. Ideally this code would provide a precise definition of the approach, easy repeatability of results, and a basis for extending the research. However, many research codebases bury high-level parameters under implementation details, are challenging to run and debug, and are difficult enough to extend that they are more likely to be rewritten. This paper describes AllenNLP, a library for applying deep learning methods to NLP research, which addresses these issues with easy-to-use command-line tools, declarative configuration-driven experiments, and modular NLP abstractions. AllenNLP has already increased the rate of research experimentation and the sharing of NLP components at the Allen Institute for Artificial Intelligence, and we are working to have the same impact across the field.")

In [14]:
[tokens for tokens in doc_1]

[Modern,
 natural,
 language,
 processing,
 (,
 NLP,
 ),
 research,
 requires,
 writing,
 code,
 .,
 Ideally,
 this,
 code,
 would,
 provide,
 a,
 precise,
 definition,
 of,
 the,
 approach,
 ,,
 easy,
 repeatability,
 of,
 results,
 ,,
 and,
 a,
 basis,
 for,
 extending,
 the,
 research,
 .,
 However,
 ,,
 many,
 research,
 codebases,
 bury,
 high,
 -,
 level,
 parameters,
 under,
 implementation,
 details,
 ,,
 are,
 challenging,
 to,
 run,
 and,
 debug,
 ,,
 and,
 are,
 difficult,
 enough,
 to,
 extend,
 that,
 they,
 are,
 more,
 likely,
 to,
 be,
 rewritten,
 .,
 This,
 paper,
 describes,
 AllenNLP,
 ,,
 a,
 library,
 for,
 applying,
 deep,
 learning,
 methods,
 to,
 NLP,
 research,
 ,,
 which,
 addresses,
 these,
 issues,
 with,
 easy,
 -,
 to,
 -,
 use,
 command,
 -,
 line,
 tools,
 ,,
 declarative,
 configuration,
 -,
 driven,
 experiments,
 ,,
 and,
 modular,
 NLP,
 abstractions,
 .,
 AllenNLP,
 has,
 already,
 increased,
 the,
 rate,
 of,
 research,
 experimentation,
 and,
 t

In [15]:
[sentences for sentences in doc_1.sents]

[Modern natural language processing (NLP) research requires writing code.,
 Ideally this code would provide a precise definition of the approach, easy repeatability of results, and a basis for extending the research.,
 However, many research codebases bury high-level parameters under implementation details, are challenging to run and debug, and are difficult enough to extend that they are more likely to be rewritten.,
 This paper describes AllenNLP, a library for applying deep learning methods to NLP research, which addresses these issues with easy-to-use command-line tools, declarative configuration-driven experiments, and modular NLP abstractions.,
 AllenNLP has already increased the rate of research experimentation and the sharing of NLP components at the Allen Institute for Artificial Intelligence, and we are working to have the same impact across the field.]

In [16]:
[x for x in doc_1.noun_chunks]

[Modern natural language processing (NLP) research,
 code,
 this code,
 a precise definition,
 the approach,
 easy repeatability,
 results,
 a basis,
 the research,
 many research codebases,
 high-level parameters,
 implementation details,
 they,
 This paper,
 AllenNLP,
 a library,
 deep learning methods,
 NLP research,
 these issues,
 easy-to-use command-line tools,
 declarative configuration-driven experiments,
 modular NLP abstractions,
 AllenNLP,
 the rate,
 research experimentation,
 the sharing,
 NLP components,
 the Allen Institute,
 Artificial Intelligence,
 we,
 the same impact,
 the field]

<h1 style="color:red">Multi Word Expression</h1>

In [17]:
# Multi word expression example 1
pattern_1 = [{'LOWER':'natural'},{'LOWER':'language'},{'LOWER':'processing'}]
patter_2 = [{'LOWER':'artificial'},{'LOWER':'intelligence'}]
matcher_1 = Matcher(nlp.vocab)
matcher_1.add('nlp_long',None,pattern_1)
matcher_1.add('AI',None,patter_2)

In [18]:
for matcher_id, start, end in matcher_1(doc_1):
    print(doc_1[start:end],'=>start of expression {}.th word and end of expression is {}.th word'.format(start,end))

natural language processing =>start of expression 1.th word and end of expression is 4.th word
Artificial Intelligence =>start of expression 135.th word and end of expression is 137.th word


In [19]:
# Multi word expression example 2
matcher = PhraseMatcher(nlp.vocab)
matcher.add('nlp_long',None,nlp('Modern natural language processing'))
matcher.add('AI',None,nlp('Artificial Intelligence'))
matcher.add('eu_clt',None,nlp('easy-to-use command-line tools'))
matcher.add('dev_conf_exp',None,nlp('declarative configuration-driven experiments'))

In [20]:
for matcher_id, start, end in matcher(doc_1):
    print(doc_1[start:end])

Modern natural language processing
easy-to-use command-line tools
declarative configuration-driven experiments
Artificial Intelligence


<h1 style="color:red">Lemmatizer</h1>

In [29]:
for tokens in doc_1:
    if tokens.pos_ =='VERB':
        print('{}\t{}'.format(tokens,tokens.lemma_))

requires	require
writing	write
would	would
provide	provide
extending	extend
bury	bury
challenging	challenge
run	run
extend	extend
rewritten	rewrite
describes	describe
applying	apply
addresses	address
use	use
driven	drive
increased	increase
working	work


In [30]:
#GPU üzerinde çalıştırma
spacy.prefer_gpu()

True